### 1. Library imports and data loading

In [5]:
# Import libraries:
import numpy as np                      # Numerical operations and array manipulation
import pandas as pd                     # Dataframe handling
import matplotlib.pyplot as plt         # Plotting
import seaborn as sns                   # Plotting

from sklearn.decomposition import PCA   # PCA plotting
from matplotlib.lines import Line2D     # Used for confidence interval plotting

In [2]:
# Load data:
df = pd.read_csv("invLAA_combined_15000_2_feature_vector_table_1000bp_subwindows.csv")
# Extract the subwindow column names from the dataframe:
feature_cols = [c for c in df.columns if c.startswith("Fst_subwin")]

### 2. Line Plot: Selected vs. Mean non-selected feature vectors

In [ ]:
# -------------------------------------------------
# 1. Compute the mean non-selected feature vector:
# -------------------------------------------------
# Selects only the subwindow Fst columns of the non-selected rows in the dataframe, 
# then calculates the mean across column. The resulting series is renamed:
mean_N = (df[df["label"] == "N"][feature_cols].mean().rename("Mean_N"))

# -------------------------------------------------
# 2. Extract selected feature vectors individually:
# -------------------------------------------------
# Extracts the subwindow Fst columns of the selected rows in the dataframe:
S_vectors = df[df["label"] == "S"][feature_cols]
# Rename the index of the selected vectors dataframe as S_1, S_2, etc. 
S_vectors.index = [f"S_{i+1}" for i in range(len(S_vectors))]

# -------------------------------------------------
# 3. Combine N & S into one dataframe for plotting:
# -------------------------------------------------
plot_df = pd.concat([mean_N.to_frame().T, S_vectors])
# Transpose so that the subwindows are on the x-axis:
plot_df = plot_df.T

# -------------------------------------------------
# 4. Create the plot:
# -------------------------------------------------
plot_df.plot(kind="line", marker="o")                       # Line plot creation
plt.xlabel("Subwindow number")                              # X-axis label
plt.ylabel("Normalized Fst")                                # Y-axis label
plt.title("Selected vs. mean non-selected feature vectors") # Title
plt.legend(title="Feature vector pattern")                  # Legend
plt.show()                                                  # Display plot

### 3. Line plot of selected vs. non-selected feature vectors

In [ ]:
# -------------------------------------------------
# 1. X-axis setup:
# -------------------------------------------------
# Creates a range for the x axis, representing the subwindow numbers:
x = range(1, len(feature_cols) + 1)

# -------------------------------------------------
# 2. Plot all individual feature vectors:
# -------------------------------------------------
# Loops through each row (window) of the dataframe to plot individual feature vectors.
for idx, row in df.iterrows():
    y = row[feature_cols].values                # Extract values from subwindow columns
    label = row["label"]                        # Extract label 
    # Plot the row in diffrent color depending on selected or not.
    if label == "N":
        plt.plot(x, y, color="blue", alpha=0.7, zorder=1)
    elif label == "S":
        plt.plot(x, y, color="gold", linewidth=3, zorder=2)

# -------------------------------------------------
# 3. Confidence interval setup & plotting:
#    Calculate the 95% confidence intervals for
#    each subwindow (column)
# -------------------------------------------------
# Initializes storage for upper and lower 95% confidence interval (CI):
ci_lower = []
ci_upper = []

# Loop through each feature column and calculate the CI for each subwindow (Fst_subwin1, Fst_subwin2, etc.):
for col in feature_cols:
    # Get the data for this feature column (subwindow):
    data = df[col].values
    # Calculate the 95% CI for the data using percentiles:
    ci = np.percentile(data, [2.5, 97.5])  # 2.5th and 97.5th percentiles for 95% CI
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])

# Plot confidence intervals:
plt.fill_between(x, ci_lower, ci_upper, color='gray', alpha=0.3, label='95% CI')

# -------------------------------------------------
# 4. Legend and labels:
# -------------------------------------------------
plt.xlabel("Subwindow Number")
plt.ylabel("Normalized Fst")
plt.ylim(0, 0.3)
plt.title("Selected vs. Neutral Feature Vectors, without breakpoints")

# Create legend manually:
legend_elements = [
    Line2D([0], [0], color='blue', lw=2, label='N'),
    Line2D([0], [0], color='gold', lw=2, label='S'),
    Line2D([0], [0], color='gray', lw=5, label='95% CI', alpha=0.3)
]
plt.legend(handles=legend_elements)

plt.show()

### 4. Heatmap of Fst Subwindows

In [ ]:
# Sets the "window_start" column as the index for the dataframe to display it on the y-axis
heatmap_df = df.set_index("window_start")

# Plot the Heatmap:
plt.figure(figsize=(10, 15))
sns.heatmap(heatmap_df[feature_cols], cmap="viridis")
plt.title("Heatmap of subwindow Fst")
plt.ylabel("Window start position along chromosome")
plt.xlabel("Subwindow number")
plt.show()

### 5. PCA Analysis of Feature Vectors

In [ ]:
# -------------------------------------------------
# 1. Prepare PCA input data:
# -------------------------------------------------
fst = df[feature_cols].values   # Extract the subwindow columns by row (windows) from the dataframe
labels = df["label"]            # Extract label

# -------------------------------------------------
# 2. Fit PCA (keep all components for scree plot):
# -------------------------------------------------
pca = PCA(n_components=8)       # Initializes PCA to reduce the data to 8 components.
coords = pca.fit_transform(fst) # Performs PCA on the feature vectors, stores the prinicple components.

# -------------------------------------------------
# 3. Add PCA coordinates to dataframe for plotting
# -------------------------------------------------
plot_df = df.copy()
plot_df["PC1"] = coords[:, 0]
plot_df["PC2"] = coords[:, 1]

# -------------------------------------------------
# 3. Plot PCA analysis:
# -------------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# -------------------------------------------------
# 3.1. PCA plot:
# -------------------------------------------------
ax = axes[0]

# Plot N first so they form the background
sns.scatterplot(
    ax=ax,
    data=plot_df[plot_df["label"] == "N"],
    x="PC1", y="PC2",
    hue="label",
    palette={"N": "#1f77b4"},
    s=40, alpha=0.9
)

# Plot S second so they appear in the foreground
sns.scatterplot(
    ax=ax,
    data=plot_df[plot_df["label"] == "S"],
    x="PC1", y="PC2",
    hue="label",
    palette={"S": "#ff7f0e"},
    s=120,
    edgecolor="black",
    linewidth=0.5
)

# Percentage of variance explained
pc1_var = pca.explained_variance_ratio_[0] * 100
pc2_var = pca.explained_variance_ratio_[1] * 100
ax.set_xlabel(f"PC1 ({pc1_var:.2f}% variance)")
ax.set_ylabel(f"PC2 ({pc2_var:.2f}% variance)")
ax.set_title("PCA of Feature Vector Table, with Breakpoints")
ax.legend(title="Label")

# -------------------------------------------------
# 3.2. Scree plot:
# -------------------------------------------------
ax2 = axes[1]

expl_var = pca.explained_variance_ratio_  # variance explained per PC

# Bar plot of explained variance
ax2.bar(np.arange(1, len(expl_var)+1), expl_var, alpha=0.3)
# Line plot for easier visual comparison
ax2.plot(np.arange(1, len(expl_var)+1), expl_var, marker="o")

ax2.set_xticks(np.arange(1, len(expl_var)+1))
ax2.set_xlabel("Principal Component")
ax2.set_ylabel("Proportion of Variance Explained")
ax2.set_title("Scree Plot of PCA Components")
ax2.grid(True, linestyle="--", alpha=0.4)

plt.tight_layout()
plt.show()